<a href="https://colab.research.google.com/github/ekacs/scrap-inaportnet-22/blob/main2/webscrap_INAPORTNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import re
from six.moves import urllib
from unicodedata import normalize
import numpy as np
import io
import datetime
from datetime import datetime, timedelta
from dateutil.tz import tzutc, tzlocal
from google.colab import files


In [ ]:
kode_lokasi='IDJKT'     #diisi kode lokasi pelabuhan, cari tahu di https://bit.ly/KODE_PELABUHAN 
tahun=2022              #diisi tahun
jenis_layanan='RKBM'    #diisi kode layanan inaportnet yg mau discrape, diakhir kode dikasi TOLONG ditambahkan tanda kurung tutup ')'
berapa_bulan=1          #diisi berapa bulan
selisih_waktu_UTC=0     # diisi waktu selisih perbedaan dg waktu UTC



#all code in here

In [ ]:

daftarbulan=['00','01','02','03','04','05','06','07','08','09','10','11','12']
urls=[]
daftarlink=[]   


for i in range(1,berapa_bulan+1):
  
  #FYI gaes syntax format URL = https://monitoring-inaportnet.dephub.go.id/site/port_detail_service?unlocode=<kode_lokasi>&bulan=<angka_bulan_2digit>&tahun=<angka_tahun_4digit>&service_code=<kode_layanan>
  urlpage = ('https://monitoring-inaportnet.dephub.go.id/site/port_detail_service?unlocode={kd_lokasi}&bulan={n}&tahun=2022&service_code={jns_produk}').format(kd_lokasi=kode_lokasi, n=(daftarbulan[i]), jns_produk=jenis_layanan)
  urls.append(urlpage)


for url in urls:
    my_url = requests.get(url) 
  
    soup = bs(my_url.content, 'html.parser')
    #print(soup)
    s = soup('a', class_='btn btn-primary ls-modal')
    #print(s)
    for link in soup.find_all('a', attrs={'href': re.compile("kode_perusahaan=")}):
      daftarlink.append('https://monitoring-inaportnet.dephub.go.id'+link.get('href')+'&download=1')
      #daftarlink.append(link.get('href'))
      #print(daftarlink)

#len(daftarlink)
#print(urls)
#print(urlpage)


hasil_scrape = pd.DataFrame(pd.read_csv(daftarlink[0]))

for i in range(0,len(daftarlink)):
  data = pd.read_html(daftarlink[i])
  df = pd.DataFrame(data[0])

  hasil_scrape = pd.concat([hasil_scrape,df],axis=0)



#hasil_scrape
#hasil_scrape.dropna(subset=["No"])


#PERHITUNGAN SELISIH WAKTU

df=hasil_scrape
df['Waktu Permohonan'] = pd.to_datetime(df['Waktu Permohonan'])
df['Waktu Respon'] = pd.to_datetime(df['Waktu Respon'])

df['waktu_permohonan_real'] = df['Waktu Permohonan']+pd.DateOffset(hours=selisih_waktu_UTC) 
df['selisih_waktu'] = df['Waktu Respon']-df['waktu_permohonan_real']
#hasil_scrape.info()



#cari data anomali Waktu Respon lebih dulu ketimbang Waktu Permohonan (asumsi ada bug di database inaportnet)

hasil_scrape_bug=hasil_scrape[hasil_scrape['selisih_waktu']<timedelta(seconds=0)]
hasil_scrape_ok=hasil_scrape[hasil_scrape['selisih_waktu']>timedelta(seconds=0)]



#make criteria in time difference
def sel(hasil_scrape):
    if hasil_scrape['selisih_waktu'] < timedelta(seconds=31):
        return "Kurang dari 30 detik"
    elif hasil_scrape['selisih_waktu'] < timedelta(minutes=30):
        return "Antara 30 detik - 30 menit"
    elif hasil_scrape['selisih_waktu'] < timedelta(hours=1):
        return "Antara 30 - 60 menit"
    elif hasil_scrape['selisih_waktu'] < timedelta(hours=6):
        return "Antara 1 - 6 jam"
    elif hasil_scrape['selisih_waktu'] < timedelta(hours=12):
        return "Antara 6 - 12 jam"
    elif hasil_scrape['selisih_waktu'] < timedelta(days=1):
        return "Antara 12 - 24 jam"
    else:
        return "Lebih dari 24 jam"

hasil_scrape_ok['lama_pelayanan'] = hasil_scrape_ok.apply(sel, axis=1)
sort_data = hasil_scrape_ok.sort_values("selisih_waktu", axis = 0, ascending = True)



lama_pelayanan = hasil_scrape_ok.groupby('lama_pelayanan')['lama_pelayanan'].count().reset_index(name='jumlah')
lama_pelayanan['persentase'] = lama_pelayanan['jumlah']/len(hasil_scrape_ok)*100
#lama_pelayanan


#gruoping month-day pelayanan

month_mapping={
    1: 'January', 
    2: 'February', 
    3: 'March', 
    4: 'April', 
    5: 'May',
    6: 'June', 
    7: 'July',
    8: 'August'
} 
dw_mapping={
    0: 'Monday', 
    1: 'Tuesday', 
    2: 'Wednesday', 
    3: 'Thursday', 
    4: 'Friday',
    5: 'Saturday', 
    6: 'Sunday'
} 


hasil_scrape_ok['bulan permohonan'] = hasil_scrape_ok['waktu_permohonan_real'].dt.month.map(month_mapping)
hasil_scrape_ok['hari permohonan'] = hasil_scrape_ok['waktu_permohonan_real'].dt.dayofweek.map(dw_mapping)

bulan_permohonan = hasil_scrape_ok.groupby('bulan permohonan')['bulan permohonan'].count().reset_index(name='jumlah')
bulan_permohonan['persentase'] = bulan_permohonan['jumlah']/len(hasil_scrape_ok)*100

def nomor(bulan_permohonan):
    if bulan_permohonan['bulan permohonan'] == 'January':
        return 1
    elif bulan_permohonan['bulan permohonan'] == 'February':
        return 2
    elif bulan_permohonan['bulan permohonan'] == 'March':
        return 3
    elif bulan_permohonan['bulan permohonan'] == 'April':
        return 4
    elif bulan_permohonan['bulan permohonan'] == 'May':
        return 5
    elif bulan_permohonan['bulan permohonan'] == 'June':
        return 6
    elif bulan_permohonan['bulan permohonan'] == 'July':
        return 7
    else:
        return 8

bulan_permohonan['nomor'] = bulan_permohonan.apply(nomor, axis=1)
bulan_permohonan = bulan_permohonan.sort_values("nomor", axis = 0, ascending = True)
#bulan_permohonan

#stat hari permohnonan

def nomor_hari(hari_permohonan):
    if hari_permohonan['hari permohonan'] == 'Monday':
        return 1
    elif hari_permohonan['hari permohonan'] == 'Tuesday':
        return 2
    elif hari_permohonan['hari permohonan'] == 'Wednesday':
        return 3
    elif hari_permohonan['hari permohonan'] == 'Thursday':
        return 4
    elif hari_permohonan['hari permohonan'] == 'Saturday':
        return 5
    elif hari_permohonan['hari permohonan'] == 'Sunday':
        return 6
    else:
        return 7
    
hari_permohonan = hasil_scrape_ok.groupby('hari permohonan')['hari permohonan'].count().reset_index(name='jumlah')
hari_permohonan['Nomor'] = hari_permohonan.apply(nomor_hari, axis=1)
hari_permohonan = hari_permohonan.sort_values("Nomor", axis = 0, ascending = True)
#hari_permohonan


#pemabgian rentang waktu permohonan
batas_waktu1 = pd.to_datetime('06:00:00', format='%H:%M:%S')
batas_waktu2 = pd.to_datetime('12:00:00', format='%H:%M:%S')
batas_waktu3 = pd.to_datetime('18:00:00', format='%H:%M:%S')

def jam(hasil_scrape_ok):
    if hasil_scrape_ok['jam permohonan'] < batas_waktu1:
        return "Jam 00.00 s.d. 06.00"
    elif hasil_scrape_ok['jam permohonan'] < batas_waktu2:
        return "Jam 06.00 s.d. 12.00"
    elif hasil_scrape_ok['jam permohonan'] < batas_waktu3:
        return "Jam 12.00 s.d. 18.00"
    else:
        return "Jam 18.00 s.d. 24.00"

hasil_scrape_ok['jam permohonan'] = hasil_scrape_ok['waktu_permohonan_real'].dt.time
hasil_scrape_ok['jam permohonan'] = pd.to_datetime(hasil_scrape_ok['jam permohonan'], format='%H:%M:%S')
hasil_scrape_ok['pembagian jam permohonan'] = hasil_scrape_ok.apply(jam, axis=1)

waktu_permohonan = hasil_scrape_ok.groupby('pembagian jam permohonan')['pembagian jam permohonan'].count().reset_index(name='jumlah')
waktu_permohonan['persentase'] = waktu_permohonan['jumlah']/len(hasil_scrape_ok)*100
#waktu_permohonan


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

#data yg anomali (asumsi ada bug di dBasenya)

In [ ]:
'''
hasil_scrape_bug.dropna(subset=['No']).to_csv('hasil_scrape_bug.csv')
files.download('hasil_scrape_bug.csv')
'''
hasil_scrape_bug.dropna(subset=['No'])

#statistik durasi lama pelayanan

In [ ]:
lama_pelayanan

,lama_pelayanan,jumlah,persentase
0,Antara 1 - 6 jam,1,0.097561
1,Antara 6 - 12 jam,1007,98.243902
2,Lebih dari 24 jam,17,1.658537


#statistik waktu pelayanan

In [ ]:
waktu_permohonan

,pembagian jam permohonan,jumlah,persentase
0,Jam 00.00 s.d. 06.00,368,35.902439
1,Jam 06.00 s.d. 12.00,440,42.926829
2,Jam 12.00 s.d. 18.00,187,18.243902
3,Jam 18.00 s.d. 24.00,30,2.926829


#statistik hari pelayanan

In [ ]:
hari_permohonan

,hari permohonan,jumlah,Nomor
1,Monday,180,1
5,Tuesday,161,2
6,Wednesday,109,3
4,Thursday,149,4
2,Saturday,140,5
3,Sunday,87,6
0,Friday,199,7


#statistik bulan pelayanan

In [ ]:
bulan_permohonan

,bulan permohonan,jumlah,persentase,nomor
3,January,668,65.170732,1
2,February,13,1.268293,2
6,March,41,4.000000,3
0,April,39,3.804878,4
7,May,33,3.219512,5
5,June,36,3.512195,6
4,July,41,4.000000,7
1,August,43,4.195122,8


#unduh hasil scrape (aslinya)

In [ ]:
#hasil scrape asli blm ada pengolahan data (termasuk didalamnya ada bug)
hasil_scrape.dropna(subset=['No']).to_csv('hasil_scrape_asli.csv')
files.download('hasil_scrape_asli.csv')

#unduh hasil scrape (+ kolom tambahan)

In [ ]:
#hasil scrape berikut kolom lainnya

#hasil_scrape_ok.head()
hasil_scrape_ok.dropna(subset=['No']).to_csv('hasil_scrape_ok.csv')
files.download('hasil_scrape_ok.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#ngintip data yg sudah diolah
hasil_scrape_ok.head()

,"<table class=""table table-bordered"">",No,Nama Kapal,Waktu Permohonan,Waktu Respon,Nomor Produk,waktu_permohonan_real,selisih_waktu,lama_pelayanan,bulan permohonan,hari permohonan,jam permohonan,pembagian jam permohonan
0,NaN,1.0,MV. SPRING GLORY,2022-10-01 02:23:52,2022-10-01 09:24:42,SL004.IDJKT.0122.000282,2022-10-01 02:23:52,0 days 07:00:50,Antara 6 - 12 jam,NaN,Saturday,1900-01-01 02:23:52,Jam 00.00 s.d. 06.00
1,NaN,2.0,MV. APOLLO SAKURA,2022-01-25 05:54:56,2022-01-25 13:06:59,SL004.IDJKT.0122.000797,2022-01-25 05:54:56,0 days 07:12:03,Antara 6 - 12 jam,January,Tuesday,1900-01-01 05:54:56,Jam 00.00 s.d. 06.00
0,NaN,1.0,MV. WORLD SPIRIT,2022-03-01 12:19:25,2022-03-01 19:28:34,SL004.IDJKT.0122.000073,2022-03-01 12:19:25,0 days 07:09:09,Antara 6 - 12 jam,March,Tuesday,1900-01-01 12:19:25,Jam 12.00 s.d. 18.00
1,NaN,2.0,MV. WORLD SPIRIT,2022-01-16 05:47:47,2022-01-16 12:49:44,SL004.IDJKT.0122.000497,2022-01-16 05:47:47,0 days 07:01:57,Antara 6 - 12 jam,January,Sunday,1900-01-01 05:47:47,Jam 00.00 s.d. 06.00
2,NaN,3.0,MV.ASTRAL ACE,2022-01-23 13:57:23,2022-01-23 20:58:41,SL004.IDJKT.0122.000732,2022-01-23 13:57:23,0 days 07:01:18,Antara 6 - 12 jam,January,Sunday,1900-01-01 13:57:23,Jam 12.00 s.d. 18.00
